# CONVERT FILES WITH DUCK_DB

In [45]:
import pandas as pd
import duckdb
import os
import time
import numpy as np
from utils import time, data_processor, constants

In [46]:
OUTPUT_PATH = r"..\\data_processed\\balkans\\"
balkans = constants.balkans

## Ownership history

In [47]:

TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed" 
TEMP_TABLE_KEY_FINANCIALS_DETAILED = "..\\data_processed\\key_financials_detailed_processed"


def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")

    COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS_DETAILED + f"\\key_financials_detailed_{country}.parquet"
    COUNTRY_FIRMOGRAPHICS = TEMP_TABLE_FIRMOGRAPHICS + f"\\firmographics_{country}.parquet" 

    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            {year} AS year,

            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            
            key_financials_detailed.operating_revenue_turnover_,
            key_financials_detailed.number_of_employees,
            key_financials_detailed.costs_of_goods_sold,
            key_financials_detailed.material_costs,
            key_financials_detailed.added_value
        FROM 
            '{path}' AS main

        LEFT JOIN 
            '{COUNTRY_KEY_FINANCIALS}' AS key_financials_detailed
        ON 
            main.subsidiary_bvd_id = key_financials_detailed.bvd_id_number
        AND
            key_financials_detailed.year = {year}

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_guo
        ON 
            main.guo_25 = firmographics_guo.bvd_id_number

        WHERE 
            main."type_of_relation" = 'GUO 25'
        AND 
            main."subsidiary_bvd_id" LIKE '{country}%'

    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df

In [49]:
@time.time_it
def fetch_and_convert_to_excel(year, country, path, output_path):
    df = get_ownership_data(year, country, path)
    os.makedirs(output_path, exist_ok=True)
    slices = data_processor.split_in_chunks(df, 5)

    for i, s in enumerate(slices):
        s.to_csv(f"{output_path}\\{country}_{year}_chunk_{i+1}.csv", index=False)
        print(f"Data for {country} - {year} - slice_{i+1} has been converted.")

In [42]:
import time
import json

errors = {}
OUTPUT_PATH = r"..\\data_processed\\balkans\\" 

total_start = time.time()
start_time = time.time()


for country in balkans:
    for year in range(2007, 2023):
        year = str(year)
        try:
            path = f"..\\data_raw\\ownership_history\\links_{year}\\*.parquet"
            fetch_and_convert_to_excel(year, country, path, OUTPUT_PATH)  
            end_time = time.time()
            duration = end_time - start_time
            print(f"Data for {country} - {year} has been converted to excel in {duration:.2f} seconds.")
        except Exception as e:
            errors[f"{country} - {year}"] = str(e)
            print(f"Error {country} - {year}")
            continue


end_time = time.time()
duration = end_time - start_time
total_end = time.time()
total_duration = total_end - total_start
print(f"Total execution time: {total_duration:.2f} seconds.")

with open("errors\\errors_balkans.json", "w") as f:
    json.dump(errors, f)

BA - 2007...
Data for BA - 2007 - slice_1 has been converted.
Data for BA - 2007 - slice_2 has been converted.
Data for BA - 2007 - slice_3 has been converted.
Data for BA - 2007 - slice_4 has been converted.
Data for BA - 2007 - slice_5 has been converted.
Execution time: 1.4423 seconds
Data for BA - 2007 has been converted to excel in 1.44 seconds.
BA - 2008...
Data for BA - 2008 - slice_1 has been converted.
Data for BA - 2008 - slice_2 has been converted.
Data for BA - 2008 - slice_3 has been converted.
Data for BA - 2008 - slice_4 has been converted.
Data for BA - 2008 - slice_5 has been converted.
Execution time: 1.2841 seconds
Data for BA - 2008 has been converted to excel in 2.73 seconds.
BA - 2009...
Data for BA - 2009 - slice_1 has been converted.
Data for BA - 2009 - slice_2 has been converted.
Data for BA - 2009 - slice_3 has been converted.
Data for BA - 2009 - slice_4 has been converted.
Data for BA - 2009 - slice_5 has been converted.
Execution time: 1.3265 seconds
Data 

In [43]:
# open errors
import json 

with open("errors/errors_balkans.json", "r") as f:
    errors = json.load(f)

errors

{}

## zip the files

In [ ]:
ZIP_OUTPOUT = "..\\zipped_files\\balkans"
data_processor.compress_csv_files(folder_to_zip=OUTPUT_PATH, zip_output=ZIP_OUTPOUT)